In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Define paths
dataset_path = '/content/drive/MyDrive/unnessery_device_dataset'

# Load and preprocess images
def load_data(data_path):
    images = []
    labels = []
    categories = ['man_without_light_on','man_without_light_off', 'man_with_light_on',  'man_light_off']
    for category in categories:
        path = os.path.join(data_path, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(128, 128))
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                img_array = img_array / 255.0
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:
                pass
    return np.array(images), np.array(labels)

images, labels = load_data(dataset_path)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Change the filepath to end with '.keras'
checkpoint = ModelCheckpoint('/content/drive/MyDrive/unnessery_device_dataset/best_model.keras',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, mode='min')

history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stop])

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.3428 - loss: 1.2503 - val_accuracy: 0.7000 - val_loss: 0.6530
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7825 - loss: 0.5261 - val_accuracy: 1.0000 - val_loss: 0.1117
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9468 - loss: 0.1825 - val_accuracy: 1.0000 - val_loss: 0.0365
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9601 - loss: 0.1340 - val_accuracy: 0.9833 - val_loss: 0.0642
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9800 - loss: 0.0484 - val_accuracy: 0.8833 - val_loss: 0.3711
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 954ms/step - accuracy: 0.9672 - loss: 0.1742 - val_accuracy: 0.9667 - val_loss: 0.0680
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9818 - loss: 0.0513 - val_accuracy: 0.9833 - val_loss: 0.0296
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9980 - loss: 0.0178 - val_accuracy: 1.0000 - val_loss: 0.0148
Epoch 

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc}')


NameError: name 'val_generator' is not defined

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/unnessery_device_dataset/best_model.keras')


In [ ]:
import os

# Check if the model file exists
model_path = '/content/drive/MyDrive/unnessery_device_dataset/best_model.keras'
if os.path.exists(model_path):
    print("Model file found!")
else:
    print("Model file not found.")


Model file found!


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/unnessery_device_dataset/best_model.keras')


In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to Google Drive
tflite_model_path = '/content/drive/MyDrive/unnessery_device_dataset/model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_path}")


Saved artifact at '/tmp/tmpfw9bbjji'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  139692032546016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032549712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032674800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032678496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032685712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032684304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032824016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032827712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032830880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139692032834400: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model sa

In [ ]:
import tensorflow as tf
from tensorflow.lite.python.interpreter import Interpreter
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the TFLite model
interpreter = Interpreter(model_path='/content/drive/MyDrive/unnessery_device_dataset/model.tflite')
interpreter.allocate_tensors()

# Get input and output details for the interpreter
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details:", input_details)
print("Output Details:", output_details)


Input Details: [{'name': 'serving_default_input_layer_2:0', 'index': 0, 'shape': array([  1, 128, 128,   3], dtype=int32), 'shape_signature': array([ -1, 128, 128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 21, 'shape': array([1, 4], dtype=int32), 'shape_signature': array([-1,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
from google.colab import files
from PIL import Image

# Upload image
uploaded = files.upload()

# Load the uploaded image
img_path = list(uploaded.keys())[0]  # Get the name of the uploaded file
img = Image.open(img_path)
img = img.resize((128, 128))  # Resize the image to the model's input size
img_array = np.array(img)

# Normalize the image
img_array = img_array / 255.0

# Add batch dimension for the prediction (the model expects a batch of images)
img_array = np.expand_dims(img_array, axis=0)

# Check the shape of the image
print("Image Shape:", img_array.shape)


Saving WhatsApp Image 2024-05-30 at 09.31.44 (1).jpeg to WhatsApp Image 2024-05-30 at 09.31.44 (1).jpeg
Image Shape: (1, 128, 128, 3)


In [ ]:
# Normalize the image and cast it to FLOAT32
img_array = img_array / 255.0
img_array = img_array.astype(np.float32)  # Convert to FLOAT32

# Add batch dimension for the prediction (the model expects a batch of images)
img_array = np.expand_dims(img_array, axis=0)

# Check the shape and dtype of the image
print("Image Shape:", img_array.shape)
print("Image Dtype:", img_array.dtype)


Image Shape: (1, 1, 128, 128, 3)
Image Dtype: float32


In [ ]:
# Check the input shape expected by the model
print("Input shape:", input_details[0]['shape'])


Input shape: [  1 128 128   3]


In [ ]:
# Resize the image to match the expected input size (128, 128, 3)
img = img.resize((128, 128))  # Resize the image to 128x128 pixels
img_array = np.array(img)  # Convert the image to a numpy array

# Normalize the image and cast to float32
img_array = img_array / 255.0
img_array = img_array.astype(np.float32)

# Add a batch dimension (model expects a batch, even if it's just one image)
img_array = np.expand_dims(img_array, axis=0)

# Check the shape and dtype of the image
print("Image Shape:", img_array.shape)  # Should print (1, 128, 128, 3)
print("Image Dtype:", img_array.dtype)  # Should be float32


Image Shape: (1, 128, 128, 3)
Image Dtype: float32


In [ ]:
# Set the input tensor with the preprocessed image
interpreter.set_tensor(input_details[0]['index'], img_array)

# Run inference
interpreter.invoke()

# Get the prediction output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Output the result
print("Prediction Output:", output_data)

# Convert the output to probabilities or classes
predicted_class = np.argmax(output_data)
print("Predicted Class:", predicted_class)

# Map the predicted class to the corresponding label
categories = ['man_with_light_on', 'man_without_light_on', 'man_with_light_off', 'man_without_light_off']
predicted_label = categories[predicted_class]
print("Predicted Label:", predicted_label)


Prediction Output: [[4.5055812e-04 9.9948502e-01 4.5386951e-06 5.9762675e-05]]
Predicted Class: 1
Predicted Label: man_without_light_on
